In [ ]:
# CEU0000000001 #National Employment (Employees in Thousands)
# BDS0000000000000000110104RQ5 #Gross Job Losses (Rate)
# JTU000000000000000LDR #Layoffs and discharges rate in total nonfarm jobs (Rate) (doesn't work and don't need it)
# CUUR0000SA0 # Consumer Price Index for all items

In [2]:
import pandas as pd
import requests
import json
from pprint import pprint
from prettytable import PrettyTable
from tqdm import tqdm
import matplotlib.pyplot as plt
from config import api_key_bls

url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = "?registrationkey-{}".format(api_key_bls)

In [3]:
#series
headers = {'Content-type': 'application/json'} #contains info about the message- such as destination
# Submit the list of series as data - payload actual data being transmitted over a network
data = json.dumps({"seriesid": ['CEU0000000001',
                                'BDS0000000000000000110104RQ5', 
                                'JTU000000000000000LDR', 
                                'CUUR0000SA0', 
                                'LNS14000000', #unemployment level
                                'LNS13000000'], #labor force level
                   'startyear':'2010', 'endyear':'2023',
                   'catalog':True,
                   'annualaverage':True,
                   "registrationkey": api_key_bls}, indent=4)
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

In [4]:
list_columns= []
for series in json_data['Results']['series']:
#     x=pd.DataFrame(columns=["series title","series id","seasonality","survey_name","measure_data_type","commerce_industy","commerce_sector",])
    seriesId = series['seriesID']
    print(seriesId)
    if 'catalog' in series.keys():
        seasonality = series['catalog']['seasonality']
        survey_name = series['catalog']['survey_name']
        measure_data_type = series['catalog']['measure_data_type']
        if 'commerce_industry' in series['catalog'].keys():
            commerce_industry = series['catalog']['commerce_industry']
        else:
            commerce_industry = None
        if 'commerce_sector' in series['catalog'].keys():
            commerce_sector = series['catalog']['commerce_sector']
        else:
            commerce_sector = None
        if 'area' in series['catalog'].keys():
            area = series['catalog']['area']
        else:
            area = None
       
    else:
        seasonality = None
        survey_name = None
        measure_data_type = None
        commerce_industry = None
        commerce_sector = None
        area= None
    #add all variables & create a list to append to the list above
    for item in series['data']:
        year = item['year']
        period = item['period']
        periodName = item['periodName']
        value = item['value']
        list_columns.append([seriesId,seasonality,survey_name,measure_data_type,
                             commerce_industry,commerce_sector,area,year,period,periodName,value])
column= pd.DataFrame(list_columns,columns=['seriesId','seasonality','survey_name','measure_data_type',
                             'commerce_industry','commerce_sector','area','year','period','periodName','value'])
column.head()

column.to_csv("BLS_Multiextract_1.csv", index=False, encoding="ISO-8859-1")
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (x.get_string())
#     output.close()

CEU0000000001
BDS0000000000000000110104RQ5
JTU000000000000000LDR
CUUR0000SA0
LNS14000000
LNS13000000
